# Setup

In [1]:
%load_ext autotime

In [2]:
import pickle
import zipfile

time: 117 ms


In [3]:
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

time: 360 ms


# Targets & IDs

Load the train names (`Id`) and labels (`Attack`):

In [4]:
df_train = pd.read_csv('../input/ytrain.csv')
print("Train shape:", df_train.shape)
df_train.head()

Train shape: (250, 2)


,Id,Attack
0,0,0
1,2,1
2,3,0
3,4,1
4,5,0


time: 23.1 ms


Load the sample submission for test names (`Id`):

In [5]:
df_test = pd.read_csv('../input/SampleSubmission.csv').drop(columns="Attack")
print("Test shape:", df_test.shape)
df_test.head()

Test shape: (250, 1)


,Id
0,1
1,6
2,7
3,8
4,10


time: 149 ms


Concatenate train and test:

In [6]:
df = pd.concat(
    [df_train, df_test],
    ignore_index=True,
)[["Id", "Attack"]]

print("Total shape:", df.shape)

Total shape: (500, 2)
time: 86.2 ms


In [7]:
df.head()

,Id,Attack
0,0,0.0
1,2,1.0
2,3,0.0
3,4,1.0
4,5,0.0


time: 114 ms


In [8]:
df.tail()

,Id,Attack
495,488,NaN
496,490,NaN
497,493,NaN
498,494,NaN
499,495,NaN


time: 108 ms


In [9]:
names = df.Id.values

time: 83.1 ms


In [10]:
y = df.Attack.values

time: 85.3 ms


# Time series

Load time series and shape them to be a 3D-tensor - with dimensions `(observations, series, points)`:

In [11]:
series = {}

for archive in '../input/xtrain.zip', '../input/xtest.zip':
    with zipfile.ZipFile(archive) as zf:
        for name in tqdm_notebook(zf.namelist(), desc=archive.split("/")[-1]):
            if name.endswith('.csv'):
                
                fname = name.split('/')[-1]
                Id = int(fname.split('.')[0])

                df = pd.read_csv(zf.open(name), header=None)
                series[Id] = df.values

HBox(children=(IntProgress(value=0, description='xtrain.zip', max=251), HTML(value='')))

HBox(children=(IntProgress(value=0, description='xtest.zip', max=251), HTML(value='')))


time: 23.6 s


In [12]:
X = np.array([series[name] for name in names]).transpose((0, 2, 1))
print("X shape:", X.shape)

X shape: (500, 56, 5760)
time: 633 ms


So there's a total of 500 observations, 56 time series for each observation and a total of 5760 time points for each time series.

# Save

Pickle the data for later use:

In [13]:
data = {
    "X": X,
    "y": y,
    "names": names,
}

time: 1.12 ms


In [14]:
with open("data.pickle", "wb") as f:
    pickle.dump(data, f)

time: 2.87 s


In [15]:
!ls -lh data.pickle

-rw-r--r-- 1 mityajj mityajj 1.3G Feb 20 23:55 data.pickle
time: 167 ms
